# Generating and simulating circuits with OpenFermion-Forest

The QubitOperator datastructure in OpenFermion is the main point of contact between OpenFermion and Forest.  Translation of the QubitOperator to `PauliTerms` and `PauliSums` is the interface that is constructed in the OpenFermion-Forest module. Fortunately, when traversing layers of abstraction in OpenFermion the QubitOperator naturally appears in all types of simulations.  Upon translation into the language of pyquil, connections to the Forest-QVM or an alternative QVM (such as reference-qvm) that understands pyquil `Program` objects can be initialized.   The following demonstration starts with the interface between the QubitOperator data structure and the `PauliTerm` and `PauliSum` data structures of pyquil and then demonstrates how to construct and simulate Hamiltonains starting from OpenFermion. 

In [1]:
from openfermion.ops import QubitOperator
from forestopenfermion import pyquilpauli_to_qubitop, qubitop_to_pyquilpauli

The interface contains two methods that mediate the translation of `PauliTerm` and `PauliSums` to `QubitOperators` and vice-versa.  

In [2]:
qubit_op = QubitOperator('X0 Y1 Z2')
pauli_term = qubitop_to_pyquilpauli(qubit_op)
print(pauli_term)

qubit_op_sum = QubitOperator('X1 Y5 Z3', coefficient=8) + QubitOperator('Y4 Z2', coefficient=1.5)
pauli_term_sum = qubitop_to_pyquilpauli(qubit_op_sum)
print(pauli_term_sum)

(1+0j)*X0*Y1*Z2
(8+0j)*X1*Z3*Y5 + (1.5+0j)*Z2*Y4


We can convert back from a `PauliSum` object ot a QubitOperator

In [3]:
reversed_term = pyquilpauli_to_qubitop(pauli_term)
print(reversed_term.isclose(qubit_op))  # should return True
reversed_sum = pyquilpauli_to_qubitop(pauli_term_sum)
print(reversed_sum.isclose(qubit_op_sum))  # should return True`

True
True


Let's generate the hopping terms for the Hubbard model Hamiltonian on four-spatial sites. 

In [4]:
from openfermion.transforms import jordan_wigner
from openfermion.ops import FermionOperator, hermitian_conjugated

In [5]:
# we'll construct the Hamiltonian terms
hopping_hamiltonian = FermionOperator()

spatial_orbitals = 4
for i in range(spatial_orbitals):
    electron_hop_alpha = FermionOperator(((2 * i, 1), (2 * ((i + 1) % spatial_orbitals), 0)))
    electron_hop_beta = FermionOperator(((2 * i + 1, 1), ((2 * ((i + 1) % spatial_orbitals) + 1), 0)))
    hopping_hamiltonian += electron_hop_alpha + hermitian_conjugated(electron_hop_alpha)
    hopping_hamiltonian += electron_hop_beta + hermitian_conjugated(electron_hop_beta)

We can turn the hopping hamiltonian into `QubitOperator` terms on ```2 * (spatial_orbital)``` qubits using the OpenFermion Jordan-Wigner routine.  openfermion-Forest provides an interface to convert the QubitOperator objects into pyquil objects and generate a Quil program from their exponentiation.  The Quil program was generated by taking each PauliTerm and converting it to a set of gates according to arXiv:1306.3991.  Once the user has data in the pyQuil format, more pyquil tools, such as a Trotterization engine, can be used.

In [6]:
from pyquil.quil import Program
from openfermionforest import exponentiate
hopping_term_generator = jordan_wigner(hopping_hamiltonian)

pyquil_program = exponentiate(hopping_term_generator)
print(pyquil_program)

H 0
H 2
CNOT 0 1
CNOT 1 2
RZ(1.0) 2
CNOT 1 2
CNOT 0 1
H 0
H 2
H 0
H 6
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
H 0
H 6
H 1
H 3
CNOT 1 2
CNOT 2 3
RZ(1.0) 3
CNOT 2 3
CNOT 1 2
H 1
H 3
H 1
H 7
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
CNOT 5 6
CNOT 6 7
RZ(1.0) 7
CNOT 6 7
CNOT 5 6
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
H 1
H 7
H 2
H 4
CNOT 2 3
CNOT 3 4
RZ(1.0) 4
CNOT 3 4
CNOT 2 3
H 2
H 4
H 3
H 5
CNOT 3 4
CNOT 4 5
RZ(1.0) 5
CNOT 4 5
CNOT 3 4
H 3
H 5
H 4
H 6
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
H 4
H 6
H 5
H 7
CNOT 5 6
CNOT 6 7
RZ(1.0) 7
CNOT 6 7
CNOT 5 6
H 5
H 7
RX(1.5707963267948966) 0
RX(1.5707963267948966) 2
CNOT 0 1
CNOT 1 2
RZ(1.0) 2
CNOT 1 2
CNOT 0 1
RX(-1.5707963267948966) 0
RX(-1.5707963267948966) 2
RX(1.5707963267948966) 0
RX(1.5707963267948966) 6
CNOT 0 1
CNOT 1 2
CNOT 2 3
CNOT 3 4
CNOT 4 5
CNOT 5 6
RZ(1.0) 6
CNOT 5 6
CNOT 4 5
CNOT 3 4
CNOT 2 3
CNOT 1 2
CNOT 0 1
RX(-1.5707963267948966) 0
RX(-1.570796326794

The returned value from exponetiate is a pyquil `Program` object.  The object has some nice features such as a dagger function, easy classical control flow construction, and introspection.  The circuit can be simulated with noise or without noise by running on the Forest-QVM or on reference-qvm.  In order to pyquil Programs on the Forest-QVM you'll need to sign up on the [Forest Home Page ](http://www.rigetti.com/forest) for a key.

In [7]:
from pyquil.api import SyncConnection
rqvm = SyncConnection()
wf, _ = rqvm.wavefunction(pyquil_program)

The resulting `Wavefunction` object from pyQuil contains pretty printing features and the ability to access the wavefunction.

In [8]:
print(wf.amplitudes)

[  7.00987799e-01 -2.45326947e-17j   3.92994831e-17 -3.93511578e-17j
   8.32667268e-17 -2.94392336e-17j   1.23259516e-32 -6.16297582e-33j
   7.28415967e-17 +1.21789046e-17j   4.29322157e-18 +4.72726933e-02j
  -2.31111593e-33 -4.62223187e-33j   6.13317367e-19 -1.73472348e-18j
   8.83177008e-17 +8.32667268e-17j  -1.54074396e-33 -3.08148791e-33j
  -1.22663473e-17 +4.72726933e-02j   1.22374427e-17 +2.56124151e-17j
   4.62223187e-33 -5.92223458e-33j  -1.73472348e-18 +3.06658683e-18j
   3.03052647e-18 +8.12841312e-18j   3.18794070e-03 +6.59316169e-18j
   5.98234944e-17 -9.24978143e-17j  -8.65320847e-02 -1.83995210e-18j
   6.16297582e-33 -3.85185989e-33j   1.73472348e-18 +3.37324552e-18j
  -7.35980840e-18 +2.49426544e-01j   3.84740581e-17 +3.01653046e-17j
   4.90653893e-18 +3.12250226e-17j   1.00148357e-32 +6.16297582e-33j
   2.31111593e-33 +4.62223187e-33j   7.35980840e-18 +6.93889390e-18j
   2.23169186e-17 -4.01382536e-17j   3.06658683e-18 -5.83548631e-03j
  -4.16333634e-17 -2.45326947e-17j

We can also pretty print the wavefunction (which is on by default) which prints the amplitudes and bitstrings in an easier to read 

In [38]:
print(wf)

(0.7009877989+0j)|00000000> + 0.0472726933j|00000101> + 0.0472726933j|00001010> + (0.0031879407+0j)|00001111> + (-0.0865320847+0j)|00010001> + 0.2494265445j|00010100> + -0.0058354863j|00011011> + (0.0168206416+0j)|00011110> + (-0.0865320847+0j)|00100010> + 0.0058354863j|00100111> + 0.2494265445j|00101000> + (-0.0168206416+0j)|00101101> + (-0.010681786+0j)|00110011> + -0.0307899779j|00110110> + 0.0307899779j|00111001> + (0.0887513323+0j)|00111100> + 0.1347657371j|01000001> + (-0.2964172847+0j)|01000100> + (-0.0090882315+0j)|01001011> + 0.0199895682j|01001110> + (-0-0.1619335007j)|01010000> + (-0.0736228577+0j)|01010101> + (0.0109203509+0j)|01011010> + 0.0049649235j|01011111> + -0.0166358961j|01100011> + (-0.0365906591+0j)|01100110> + (-0.0479525495+0j)|01101001> + (-0-0.105471649j)|01101100> + -0.0199895682j|01110010> + (0.0090882315+0j)|01110111> + (-0.0576194244+0j)|01111000> + 0.0261965972j|01111101> + 0.1347657371j|10000010> + (0.0090882315+0j)|10000111> + (-0.2964172847+0j)|1000100